<a href="https://colab.research.google.com/github/weathon/3d2smile/blob/main/thesis_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget file.weasoft.com/images.zip
!wget file.weasoft.com/summary.csv
!git clone https://github.com/suanfaxiaohuo/SwinOCSR.git
!pip install focal_loss_torch wandb
!pip3 install deepsmiles yacs tqdm
!wget file.weasoft.com/model.py -O model.py
!wget file.weasoft.com/reverse.map

In [5]:
from model import *

--2024-03-18 23:44:17--  http://file.weasoft.com/model.py
Resolving file.weasoft.com (file.weasoft.com)... 149.28.13.194
Connecting to file.weasoft.com (file.weasoft.com)|149.28.13.194|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3535 (3.5K) [text/x-python]
Saving to: ‘model.py’

model.py            100%[===================>]   3.45K  --.-KB/s    in 0s      

2024-03-18 23:44:17 (48.2 MB/s) - ‘model.py’ saved [3535/3535]



In [8]:
import torch, torchvision
if torch.cuda.is_available():
    device = torch.device('cuda:0')
else:
    device = torch.device('cpu')
mod = torch.load("drive/MyDrive/final.with.test.7.pt", map_location=device)

In [14]:
t = 1
import deepsmiles
from PIL import Image
import json


converter = deepsmiles.Converter(rings=True, branches=True)
def triangle_mask(size):
    mask = 1- np.triu(np.ones((1, size, size)),k=1).astype('uint8')
    mask = torch.autograd.Variable(torch.from_numpy(mask))
    return mask

def top_k_2d(m, k):
  values, indices = torch.topk(m.flatten(), k)
  return indices//m.shape[1], indices%m.shape[1]

def pad_pack(sequences):
    maxlen = max(map(len, sequences))
    batch = torch.LongTensor(len(sequences),maxlen).fill_(0)
    for i,x in enumerate(sequences):
        batch[i,:len(x)] = torch.LongTensor(x)
    return batch, maxlen

reversed_word_map = {}

with open("reverse.map","r") as f:
  reversed_word_map = json.loads(f.read())
reversed_word_map_={}

for i in reversed_word_map.keys():
  reversed_word_map_[int(i)] = reversed_word_map[i]


In [18]:
import numpy as np
class SMILESGenerator(torch.nn.Module):
  def __init__(self, encoder, decoder):
    super().__init__()
    self.encoder = encoder
    self.decoder = decoder

  def predict_next(self, image_embeding, text_in_):
    pass

  def forward(self, images, text_in_, max_len, beam, r=0.05): #just changed beam=1 and it runs so beam cannot be random number???  yeah it has to been factor of 676, which is 2 2 13 13 but original paper did not use beam search
    with torch.no_grad():
      image_feature = []
      for i in images:
        image_feature.append(self.encoder(i))
      image_feature = torch.stack(image_feature)
      image_feature = image_feature.squeeze(1)
      mem = gen.decoder.encoder(gen.decoder.encoder_dim(image_feature))
      # mem = torch.mean(mem, dim=0)
      # mem = mem.unsqueeze(0).repeat_interleave(beam, dim=0)
      mem = mem.repeat_interleave(beam, dim=0)
      top_n = torch.tensor(text_in_)
      top_n_p = torch.tensor([0.]).to(device)
      for i in range(max_len):
        padded_text, l = pad_pack(top_n)
        padded_text = padded_text.to(device)
        out = torch.nn.functional.softmax(self.decoder.decoder(padded_text, mem, x_mask=triangle_mask(l).to(device))[:,-1,:].squeeze(dim=1), dim=1)
        reward = torch.zeros(top_n_p.shape)
        if i>1:
          for j in range(beam):
            if top_n[j][-1]==78:
              out[j]=torch.zeros(79)
              out[j][78]=1
              reward[j]=(0.)
            else:
              reward[j]=(r)
        top_n_p+=reward.to(device)
        tmp_n = top_n_p.repeat((79,1)).T+torch.log(out)
        x, y = top_k_2d(tmp_n, beam)
        new_top_n = []
        new_top_n_p = []
        for j in range(beam):
          new_top_n.append(torch.cat((top_n[x[j]], y[j].cpu().unsqueeze(-1))))
          new_top_n_p += [tmp_n[x[j], y[j]]]
        top_n = new_top_n
        top_n_p = torch.tensor(new_top_n_p).to(device)
        count = 0
        for i in top_n:
          if 78 in i:
            count+=1
        if count==beam:
          return top_n, top_n_p
      return top_n, top_n_p
mod = mod.eval()
gen = SMILESGenerator(mod.encoder, mod.decoder)
img1 = torch.tensor(np.array(Image.open("111.png").convert("RGB").resize((400,400)))).unsqueeze(0).permute(0,3,1,2).to(device).to(torch.float32)
gen = gen.train(False) #forgot gen=
res = gen.forward([img1], [[77]], 100, 10, 0)
print()
for i in res[0]:
  print(converter.decode("".join([reversed_word_map_[i] for i in i.numpy()])))
print(torch.exp(res[1]))


<start>COCCN1CC1<end><end><end><end><end>
<start>COCN1CC1<end><end><end><end><end><end>
<start>COCN1CCC1<end><end><end><end><end>
<start>COCN1CCNC1<end><end><end><end>
<start>COCCN1CCC1<end><end><end><end>
<start>COCN1C=CNC1<end><end><end>
<start>COCN1C=NC1<end><end><end><end>
<start>COCCN1C=CNC1<end><end>
<start>COCCN1C=CC1<end><end><end>
<start>COCCN1C=CN=C1<end>
tensor([0.1362, 0.1031, 0.0239, 0.0212, 0.0210, 0.0205, 0.0193, 0.0158, 0.0139,
        0.0095])
